In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


C:\Users\aakas\AppData\Local\Temp\ipykernel_11908\455212364.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wine_data = load_wine()
X= wine_data.data
y = wine_data.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
np.arange(2, 20,2)

array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])

In [12]:
param_dist = {
    "criterion": ["gini", "entropy"],
    "max_depth": [1,3,5,7,9,11,13],
    "min_samples_split": np.arange(2, 20,2),
    "min_samples_leaf": np.arange(2, 20,2),
    "max_features": ["auto", "sqrt", "log2", None]}
dt_classifier = DecisionTreeClassifier()
random_search = RandomizedSearchCV(dt_classifier, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

d:\Machine_learning_projects\env\lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
120 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Machine_learning_projects\env\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Machine_learning_projects\env\lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "d:\Machine_learning_projects\env\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "d:\Machi

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 3, 5, 7, 9, 11, 13],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
                   random_state=42, scoring='accuracy')

In [13]:
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 3, 'criterion': 'gini'}


In [14]:
best_dt_classifier = DecisionTreeClassifier(**best_params)
best_dt_classifier.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=2)

In [15]:
y_pred = best_dt_classifier.predict(X_test)

In [16]:
accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy:", accuracy)

Test Set Accuracy: 0.9444444444444444


Grow a random forest using the following steps:-

  1. Continuing the previous question, create 10 subsets of the training dataset. You can use the ShuffleSplit                class for it.
  2. Train 1 decision tree on each subset, using the best hyperparameter values found in the previous question.
  3. Evaluate all the trees on the test dataset. Are they performing better than the tree created in the previous question?

In [17]:
from sklearn.model_selection import ShuffleSplit
from sklearn.base import clone

In [18]:
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
subset_indices = list(shuffle_split.split(X_train))

In [19]:
decision_trees = []
for train_index, _ in subset_indices:
    subset_X_train = X_train[train_index]
    subset_y_train = y_train[train_index]
    dt = DecisionTreeClassifier(**best_params)
    dt.fit(subset_X_train, subset_y_train)
    decision_trees.append(dt)



In [20]:
test_predictions = []
for dt in decision_trees:
    test_predictions.append(dt.predict(X_test))



In [21]:
accuracies = []
for i, pred in enumerate(test_predictions):
    accuracy = accuracy_score(y_test, pred)
    accuracies.append(accuracy)
    print(f"Decision Tree {i+1} Accuracy:", accuracy)



Decision Tree 1 Accuracy: 0.9722222222222222
Decision Tree 2 Accuracy: 0.9722222222222222
Decision Tree 3 Accuracy: 0.8611111111111112
Decision Tree 4 Accuracy: 0.9444444444444444
Decision Tree 5 Accuracy: 0.9444444444444444
Decision Tree 6 Accuracy: 0.8888888888888888
Decision Tree 7 Accuracy: 0.9444444444444444
Decision Tree 8 Accuracy: 0.9722222222222222
Decision Tree 9 Accuracy: 0.9166666666666666
Decision Tree 10 Accuracy: 0.9444444444444444


In [22]:
average_accuracy = sum(accuracies) / len(accuracies)
print("Average Accuracy of Random Forest:", average_accuracy)

Average Accuracy of Random Forest: 0.9361111111111112


### Conclusion:
some threes are performing better then our tess result but some of them performing simmilar or not well in compare to tree. But when we take average of all trees then it gives accuracy less then the result what we get from decisionTreeClassfier.